In [1]:
import pandas as pd
import numpy as np
import json
import utils
import tqdm

metrics: \
sqtring matching score \
executability \
execution score  

In [2]:
with open('spider_data/train_spider.json') as f:
    query_data = json.load(f)

In [3]:
with open('spider_data/tables.json') as f:
    table_data = json.load(f)

In [4]:
table_data_dict = {rec['db_id']:rec for rec in table_data}

In [5]:
def get_schema(query, table_data_dict):
    db = query['db_id']
    tbl_id = query['sql']['from']['table_units'][0][1]
    table_name = table_data_dict[db]['table_names_original'][tbl_id]

    columns = []
    for names,type in zip(table_data_dict[db]['column_names_original'],table_data_dict[db]['column_types']):
        if names[0]==tbl_id:
            columns.append((names[1], type))
    
    tbl_details = {
        table_name:columns
    }
    
    df = pd.DataFrame(columns, columns = ['name','type'])
    df['comment'] = np.nan
    tbl_schema = utils.format_topk_sql(tbl_details, shuffle=False)

    return tbl_schema, df, table_name

In [6]:
single_table_queries = [query for query in query_data if len(query['sql']['from']['table_units'])==1]

In [ ]:
prompt_template = """ 
                        ### Instructions:
                        Your task is to convert a question into a SQL query, given a Postgres database schema.
                        Adhere to these rules:
                        - **Deliberately go through the question and database schema word by word** to appropriately answer the question
                        - **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
                        - When creating a ratio, always cast the numerator as float
                        
                        ### Input:
                        Generate a SQL query that answers the question {question}.
                        This query will run on a database whose schema is represented in this string:
                        {db_schema}
                        
                        
                        ### Response:
                        Based on your instructions, here is the SQL query I have generated to answer the question {question}:
                        ```sql
                        """

In [60]:
model_name = 'sqlcoder'

sql_query, prompt = utils.getModelResult(tbl_schema, query['question'], model_name, table_name, df)

Running pre-processing...
Pruned schema: 
CREATE TABLE head (
  age DECIMAL(38, 0), --None
  head_ID DECIMAL(38, 0), --None
  name TEXT, --None
  born_state TEXT, --None
);



CREATE TABLE head (
  age DECIMAL(38, 0), --None
  head_ID DECIMAL(38, 0), --None
  name TEXT, --None
  born_state TEXT, --None
);


prompt:  
                        ### Instructions:
                        Your task is to convert a question into a SQL query, given a Postgres database schema.
                        Adhere to these rules:
                        - **Deliberately go through the question and database schema word by word** to appropriately answer the question
                        - **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
                        - When creating a ratio, always cast the numerator as float
                        
                        ### Input:
                        Generate 

In [16]:
from importlib import reload
reload(utils)

<module 'utils' from '/workspace/codegen/codegen/utils.py'>

In [9]:
import requests

OLLAMA_URL = 'http://127.0.0.1:11434'
class OLLAMA:
    def __init__(self, OLLAMA_URL, model_name):
        self.model_name = model_name
        self.ollama_url = OLLAMA_URL
        self.ollama_endpoint = '/api/generate'

    def run(self, prompt):
        data = {
            'model': self.model_name,
            'prompt': prompt,
            'stream': False,
            "options":{"temperature":0.1}
        }

        headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        }

        resp = requests.post(url = f'{self.ollama_url}{self.ollama_endpoint}',
                             data = json.dumps(data),
                             headers = headers)
        query = resp.json()['response']
        # print(f'JSON resp: {query}')
        return query

In [20]:
model_name = 'sqlcoder:7b'
prompt = prompt_template.format(question=single_table_queries[0]['question'], db_schema=tbl_schema)
ollama = OLLAMA(OLLAMA_URL=utils.OLLAMA_URL, model_name=model_name)
generated_query, resp = ollama.run(prompt)

JSON resp:  SELECT COUNT(*) FROM head WHERE age > 56;
                        ```


In [10]:
model_name = 'sqlcoder:7b'
results = []
i=0
for query in tqdm.tqdm(single_table_queries):
    i+=1
    try:
        tbl_schema, _, _ = get_schema(query, table_data_dict)
        
        prompt = prompt_template.format(question=query['question'], db_schema=tbl_schema)
        ollama = OLLAMA(OLLAMA_URL=utils.OLLAMA_URL, model_name=model_name)
        generated_query = ollama.run(prompt)
    
        results.append(generated_query)
    except:
        results.append('failure\n')

    if i%5==0:
        with open('results.txt', 'a') as f:
            for rec in results:
                if '\n' in rec:
                    f.write(rec)
                else:
                    f.write(f"{rec}\n")

100%|██████████| 4361/4361 [1:46:23<00:00,  1.46s/it]  


In [10]:
tbl_schema, df, table_name = get_schema(single_table_queries[0], table_data_dict)

In [15]:
len(df)

4

In [7]:
model_name = 'sqlcoder:7b'
results = []
i=0
for query in tqdm.tqdm(single_table_queries):
    i+=1
    try:
        tbl_schema, df, table_name = get_schema(query, table_data_dict)

        sql_query, prompt = utils.getModelResult(tbl_schema, query['question'], model_name, table_name, df)
    
        results.append(sql_query)
    except:
        results.append('failure\n')
        
    # if i%50==0:
    #     with open('results_codegen.txt', 'a') as f:
    #         for rec in results:
    #             if '\n' in rec:
    #                 f.write(rec)
    #             else:
    #                 f.write(f"{rec}\n")
    #     results = []

# with open('results_codegen.txt', 'a') as f:
#     for rec in results:
#         if '\n' in rec:
#             f.write(rec)
#         else:
#             f.write(f"{rec}\n")

  0%|          | 0/4361 [00:00<?, ?it/s]

Running pre-processing...


  0%|          | 1/4361 [00:00<55:40,  1.31it/s]

Running pre-processing...


  0%|          | 2/4361 [00:01<1:08:14,  1.06it/s]

...

100%|█████████▉| 4359/4361 [1:45:21<00:03,  1.59s/it]

Running pre-processing...


100%|█████████▉| 4360/4361 [1:45:22<00:01,  1.58s/it]

Running pre-processing...


100%|██████████| 4361/4361 [1:45:24<00:00,  1.45s/it]


In [8]:
len(results)

4361

In [9]:
import pickle

In [10]:
with open('results_codegen.pkl', 'wb') as f:
    pickle.dump(results,f)

In [11]:
with open('results_codegen.pkl', 'rb') as f:
    ls = pickle.load(f)